In [7]:
import pandas as pd 

path = 'reg_data/multi_reg.csv'
df = pd.read_csv(path)
df = df[['batch_size', 'learning_rate','dropout', 'val_loss']].dropna()
df

,batch_size,learning_rate,dropout,val_loss
1,83,0.007722,0.160347,0.055907
2,77,0.005789,0.115295,0.057823
3,97,0.008519,0.662069,0.053928
4,79,0.002623,0.194170,0.054212
5,98,0.001041,0.450008,0.052145
...,...,...,...,...
107,18,0.002310,0.599089,0.054677
108,45,0.001917,0.303026,0.056951
109,79,0.003171,0.297139,0.055611
110,11,0.006461,0.197863,0.057371


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
standard_df = scaler.fit_transform(df)
standard_df = pd.DataFrame(standard_df, columns=df.columns)
standard_df

,batch_size,learning_rate,dropout,val_loss
0,0.430945,0.883276,-1.392001,-0.000305
1,0.245630,0.208520,-1.642189,0.296699
2,0.863346,1.161393,1.394240,-0.307078
3,0.307401,-0.896859,-1.204170,-0.263088
4,0.894232,-1.449036,0.216590,-0.583520
...,...,...,...,...
101,-1.576634,-1.006183,1.044494,-0.190998
102,-0.742717,-1.143212,-0.599651,0.161502
103,0.307401,-0.705659,-0.632347,-0.046216
104,-1.792835,0.443122,-1.183661,0.226712


In [13]:
min_df = standard_df[standard_df['val_loss'] < -1.8]
min_df

,batch_size,learning_rate,dropout,val_loss
62,0.554488,-1.396737,0.989011,-1.893608
63,1.419291,-0.732733,0.976381,-1.809903
64,0.307401,1.676200,1.262199,-1.834054
67,0.245630,-0.208953,0.501417,-1.939128
68,-0.495630,-1.245971,-0.097575,-1.907281
69,-1.638406,-0.223779,-0.031655,-1.895419
71,0.986890,1.425300,0.597900,-1.881392
72,-0.372087,1.070993,0.619954,-1.809705
74,-0.773603,1.595197,-0.231895,-1.902011


In [29]:
import plotly.express as px 
import plotly.graph_objects as go 

fig = go.Figure()
fig.add_traces(
    go.Mesh3d(
        x=standard_df['learning_rate'], y=standard_df['dropout'], z=standard_df['val_loss'], 
        opacity=0.8,
        intensity=standard_df['batch_size'], colorbar_title='batch_size'
        )
    )
fig.add_traces(
    go.Scatter3d(
        x=min_df['learning_rate'], y=min_df['dropout'], z=min_df['val_loss']
    )
)
fig.update_layout(
    scene=dict(
        xaxis_title='learning_rate',
        yaxis_title='dropout',
        zaxis_title='val_loss',
    )
)
fig.show()

In [23]:
fig = px.scatter(standard_df, x=standard_df['learning_rate'], y=standard_df['val_loss'])
fig.show()